In [ ]:
# Best performance model has been saved to: [{'chexpert-classification_auc': 'chexpert-classification_val_auc_0_6563_test_auc_0_6099_epoch16_08-01-2023 09-53-30_chexpert_cl_fix'}]
# The final model has been saved to: [chexpert-classification_val_auc_0_6147_test_auc_0_6142_epoch200_08-01-2023 15-06-12_chexpert_cl_fix]

# Best performance model has been saved to: [{'chexpert-classification_auc': 'chexpert-classification_val_auc_0_7130_test_auc_0_7087_epoch114_08-01-2023 18-10-54_chexpert_baseline'}]
# The final model has been saved to: [chexpert-classification_val_auc_0_6837_test_auc_0_6835_epoch200_08-01-2023 20-19-54_chexpert_baseline]

# Best performance model has been saved to: [{'chexpert-classification_auc': 'chexpert-classification_val_auc_0_7169_test_auc_0_7275_epoch168_08-02-2023 00-53-40_chexpert_cl_no_fix'}]
# The final model has been saved to: [chexpert-classification_val_auc_0_6988_test_auc_0_7129_epoch200_08-02-2023 02-00-43_chexpert_cl_no_fix]


In [2]:
import pandas as pd
import numpy as np
import torch
import utils.print as print_f
import os

from utils.engine import evaluate, get_iou_types
from utils.plot import plot_losses, plot_performance
from data.strs import TaskStrs, SourceStrs, FusionStrs

from models.utils import get_model_size_in_MB
from models.build import create_model_from_setup
from models.setup import ModelSetup
from models.train import TrainingInfo, get_task_metric_name_from_standard
from utils.save import check_best, end_train, get_data_from_metric_logger
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_performance
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup, get_coco_eval_params, get_dynamic_loss
from utils.init import reproducibility, clean_memory_get_device
from data.paths import MIMIC_EYE_PATH
# from datetime import datetime
# from models.dynamic_loss import DynamicWeightedLoss
from utils.engine import train_one_epoch, evaluate
from torch import optim
from models.setup import ModelSetup
from data.paths import MIMIC_EYE_PATH, PHYSIONET_PATH
from models.load import get_trained_model
from models.load import load_cl_pretrained


# from data.load import seed_worker, get_dataloader_g

## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

from models.components.task_performers import ObjectDetectionPerformer
from data.constants import DEFAULT_MIMIC_CLINICAL_NUM_COLS, DEFAULT_MIMIC_CLINICAL_CAT_COLS

%matplotlib inline

In [1]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
# # 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
# device = clean_memory_get_device()
# reproducibility()
# if torch.cuda.is_available():
#     print(torch.cuda.memory_summary())

In [3]:
get_epoch_from_model = lambda select_model: int(([substr for substr in select_model.value.split("_") if "epoch" in substr][0]).replace("epoch", ""))
get_model_name = lambda select_model: str(select_model).split(".")[-1]

In [5]:
import pickle
# only need to load train_info
def load_train_info(model_path):
    with open(os.path.join("training_records", f"{model_path}.pkl"), "rb") as f:
        train_info: TrainingInfo = pickle.load(f)
    return train_info    

In [6]:
baseline_64 = "chexpert-classification_val_auc_0_6837_test_auc_0_6835_epoch200_08-01-2023 20-19-54_chexpert_baseline"
cl_64 = "chexpert-classification_val_auc_0_6988_test_auc_0_7129_epoch200_08-02-2023 02-00-43_chexpert_cl_no_fix"
cl_64_fix = "chexpert-classification_val_auc_0_6147_test_auc_0_6142_epoch200_08-01-2023 15-06-12_chexpert_cl_fix"

In [9]:
baseline_64_t = load_train_info(baseline_64)
cl_64_t = load_train_info(cl_64)
cl_64_fix_t = load_train_info(cl_64_fix)

In [10]:
def get_best_epoch_from_t(t):
    for e in t.best_performance_model_path['chexpert-classification_auc'].split("_"):
        if "epoch" in e:
            return int(e.replace("epoch",""))

In [8]:
baseline_t

In [11]:
for t in [baseline_64_t, cl_64_t, cl_64_fix_t]:
    print(t.model_setup.name)
    best_epoch = get_best_epoch_from_t(t)
    print(f"best at {best_epoch} epoch")
    print(t.performance['test'][best_epoch-1])

chexpert_baseline
best at 114 epoch
{'chexpert-classification': {'f1': 0.5461847389558233, 'precision': 0.7391304347826086, 'accuracy': 0.9327380952380953, 'recall': 0.43312101910828027, 'auc': 0.7086813237366745}}
chexpert_cl_no_fix
best at 168 epoch
{'chexpert-classification': {'f1': 0.5781250000000001, 'precision': 0.7474747474747475, 'accuracy': 0.9357142857142857, 'recall': 0.4713375796178344, 'auc': 0.7274613045823906}}
chexpert_cl_fix
best at 16 epoch
{'chexpert-classification': {'f1': 0.3318777292576419, 'precision': 0.5277777777777778, 'accuracy': 0.9089285714285714, 'recall': 0.24203821656050956, 'auc': 0.6098569283721786}}
